# LAB Monday: Scrape the current top 100 songs and their respective artists, and put the information into a pandas dataframe.

In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
url='https://www.billboard.com/charts/hot-100/'


In [3]:
results=requests.get(url, headers ={'Accept-Language':'en-US'})


In [4]:
soup = BeautifulSoup(results.content,'html.parser')


In [5]:
soup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)


'Anti-Hero'

In [6]:
soup.select('span.c-label.a-no-trucate')[0].get_text(strip=True)


'Taylor Swift'

In [7]:
title=[]
artists=[]

len_songs=len(soup.select('h3.c-title.a-no-trucate'))

In [8]:
len_songs


100

In [9]:
# title_df=[]
# artist_df=[]

# for songs in title:
#     title_df.append(songs.replace('\n', ''))

# for songs in artists:
#     artist_df.append(songs.replace('\n', ''))


# using strip=True in the step before it is not necessary to replace \n

In [10]:
len(title)


0

In [11]:
len(artists)


0

In [12]:
import pandas as pd

songs=pd.DataFrame({'Song_title':title, 'Artists':artists})
songs


,Song_title,Artists


In [13]:
import pandas as pd

songs=pd.DataFrame({'Song_title':title, 'Artists':artists})
songs


,Song_title,Artists


In [14]:
songs

,Song_title,Artists


# LAB Wednesday: API wrappers - Create your collection of songs & audio features

In [15]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

In [16]:
client_id = str(getpass.getpass('client_id?'))
client_secret = str(getpass.getpass('client_secret?'))

client_id?········
client_secret?········


In [17]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [18]:
playlist=sp.user_playlist_tracks('Bradnumber1','1YL4XoegERoragv0RK2RC9')


In [19]:
playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [20]:
playlist['items'][4]['track']['id']

'1Bt06pqlg8pqlpnl4R6Rd2'

In [21]:
playlist['total']

9994

In [22]:
len(playlist['items'])

100

In [23]:
def get_playlist_tracks(user_id,playlist_id):
    results = sp.user_playlist_tracks(user_id,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
    tracks.extend(results['items'])
    return tracks

In [24]:
full_track_ids=get_playlist_tracks('Bradnumber1','1YL4XoegERoragv0RK2RC9')

In [25]:
len(full_track_ids)

194

In [26]:
full_track_ids[0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [27]:

full_track_ids[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [28]:
full_track_ids[0]['track']['name']


'What Is Truth'

In [29]:
full_track_ids[0]['track']['artists'][0]['name']


'Johnny Cash'

In [30]:
full_track_ids[0]['track']['id']


'0snyjt8NMg7Dc7E3xnDH9b'

In [31]:
title=[]
artist=[]
song_id=[]

for i in full_track_ids:
    title.append(i['track']['name'])
    artist.append(i['track']['artists'][0]['name'])
    song_id.append(i['track']['id'])

In [32]:
playlist_df=pd.DataFrame({'title': title, 'artist': artist, 'song_id': song_id})


In [33]:
playlist_df


,title,artist,song_id
0,What Is Truth,Johnny Cash,0snyjt8NMg7Dc7E3xnDH9b
1,The Boxer,Simon & Garfunkel,76TZCvJ8GitQ2FA1q5dKu0
2,You Are The Reason - Duet Version,Calum Scott,4gJAWJnE5cvAAWyTMTxgdY
3,Genie in a Bottle,Christina Aguilera,2EVv4Vxu3HGPr32CQecgHx
4,Those Kinda Nights (feat. Ed Sheeran),Eminem,1Bt06pqlg8pqlpnl4R6Rd2
...,...,...,...
189,Sand in My Shoes,Dido,4ObePnl5sf02pKfYUeGpkp
190,Turn Me On,Norah Jones,1qzHqfvKrZWo6dVHM1dXrj
191,Made You Look,Meghan Trainor,0QHEIqNKsMoOY5urbzN48u
192,2 Be Loved (Am I Ready),Lizzo,13Irp51zj01BZu2XtDrnAg


In [34]:
spaceplaylist=[]
for i in range(len(playlist_df)):
    spaceplaylist.extend(sp.audio_features(tracks=playlist_df['song_id'][i]))

In [35]:
song_features=pd.DataFrame.from_dict(spaceplaylist)


In [36]:
song_features


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.681,0.385,4,-12.568,1,0.1180,0.38300,0.171000,0.2770,0.628,86.137,audio_features,0snyjt8NMg7Dc7E3xnDH9b,spotify:track:0snyjt8NMg7Dc7E3xnDH9b,https://api.spotify.com/v1/tracks/0snyjt8NMg7D...,https://api.spotify.com/v1/audio-analysis/0sny...,157667,4
1,0.439,0.488,11,-14.464,1,0.0615,0.70200,0.000339,0.1600,0.629,93.017,audio_features,76TZCvJ8GitQ2FA1q5dKu0,spotify:track:76TZCvJ8GitQ2FA1q5dKu0,https://api.spotify.com/v1/tracks/76TZCvJ8GitQ...,https://api.spotify.com/v1/audio-analysis/76TZ...,308520,4
2,0.379,0.329,10,-8.796,1,0.0301,0.44700,0.000000,0.1480,0.127,171.090,audio_features,4gJAWJnE5cvAAWyTMTxgdY,spotify:track:4gJAWJnE5cvAAWyTMTxgdY,https://api.spotify.com/v1/tracks/4gJAWJnE5cvA...,https://api.spotify.com/v1/audio-analysis/4gJA...,190300,3
3,0.627,0.800,1,-6.945,1,0.1620,0.21000,0.000123,0.1370,0.912,175.815,audio_features,2EVv4Vxu3HGPr32CQecgHx,spotify:track:2EVv4Vxu3HGPr32CQecgHx,https://api.spotify.com/v1/tracks/2EVv4Vxu3HGP...,https://api.spotify.com/v1/audio-analysis/2EVv...,217573,4
4,0.813,0.641,4,-6.279,0,0.3370,0.02690,0.000000,0.0864,0.390,103.827,audio_features,1Bt06pqlg8pqlpnl4R6Rd2,spotify:track:1Bt06pqlg8pqlpnl4R6Rd2,https://api.spotify.com/v1/tracks/1Bt06pqlg8pq...,https://api.spotify.com/v1/audio-analysis/1Bt0...,177693,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.633,0.809,11,-6.702,1,0.0317,0.11400,0.000102,0.1110,0.267,125.953,audio_features,4ObePnl5sf02pKfYUeGpkp,spotify:track:4ObePnl5sf02pKfYUeGpkp,https://api.spotify.com/v1/tracks/4ObePnl5sf02...,https://api.spotify.com/v1/audio-analysis/4Obe...,299173,4
190,0.429,0.180,10,-10.694,1,0.0551,0.86400,0.000004,0.1040,0.485,170.645,audio_features,1qzHqfvKrZWo6dVHM1dXrj,spotify:track:1qzHqfvKrZWo6dVHM1dXrj,https://api.spotify.com/v1/tracks/1qzHqfvKrZWo...,https://api.spotify.com/v1/audio-analysis/1qzH...,154800,3
191,0.838,0.525,10,-3.562,1,0.0665,0.34500,0.000002,0.0771,0.884,144.981,audio_features,0QHEIqNKsMoOY5urbzN48u,spotify:track:0QHEIqNKsMoOY5urbzN48u,https://api.spotify.com/v1/tracks/0QHEIqNKsMoO...,https://api.spotify.com/v1/audio-analysis/0QHE...,134256,4
192,0.721,0.769,7,-4.111,1,0.1050,0.09220,0.000000,0.0817,0.915,155.932,audio_features,13Irp51zj01BZu2XtDrnAg,spotify:track:13Irp51zj01BZu2XtDrnAg,https://api.spotify.com/v1/tracks/13Irp51zj01B...,https://api.spotify.com/v1/audio-analysis/13Ir...,187108,4


In [37]:
playlist_features = playlist_df.merge(song_features, left_on='song_id', right_on='id')


In [38]:
playlist_features


,title,artist,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,What Is Truth,Johnny Cash,0snyjt8NMg7Dc7E3xnDH9b,0.681,0.385,4,-12.568,1,0.1180,0.38300,...,0.2770,0.628,86.137,audio_features,0snyjt8NMg7Dc7E3xnDH9b,spotify:track:0snyjt8NMg7Dc7E3xnDH9b,https://api.spotify.com/v1/tracks/0snyjt8NMg7D...,https://api.spotify.com/v1/audio-analysis/0sny...,157667,4
1,The Boxer,Simon & Garfunkel,76TZCvJ8GitQ2FA1q5dKu0,0.439,0.488,11,-14.464,1,0.0615,0.70200,...,0.1600,0.629,93.017,audio_features,76TZCvJ8GitQ2FA1q5dKu0,spotify:track:76TZCvJ8GitQ2FA1q5dKu0,https://api.spotify.com/v1/tracks/76TZCvJ8GitQ...,https://api.spotify.com/v1/audio-analysis/76TZ...,308520,4
2,You Are The Reason - Duet Version,Calum Scott,4gJAWJnE5cvAAWyTMTxgdY,0.379,0.329,10,-8.796,1,0.0301,0.44700,...,0.1480,0.127,171.090,audio_features,4gJAWJnE5cvAAWyTMTxgdY,spotify:track:4gJAWJnE5cvAAWyTMTxgdY,https://api.spotify.com/v1/tracks/4gJAWJnE5cvA...,https://api.spotify.com/v1/audio-analysis/4gJA...,190300,3
3,Genie in a Bottle,Christina Aguilera,2EVv4Vxu3HGPr32CQecgHx,0.627,0.800,1,-6.945,1,0.1620,0.21000,...,0.1370,0.912,175.815,audio_features,2EVv4Vxu3HGPr32CQecgHx,spotify:track:2EVv4Vxu3HGPr32CQecgHx,https://api.spotify.com/v1/tracks/2EVv4Vxu3HGP...,https://api.spotify.com/v1/audio-analysis/2EVv...,217573,4
4,Those Kinda Nights (feat. Ed Sheeran),Eminem,1Bt06pqlg8pqlpnl4R6Rd2,0.813,0.641,4,-6.279,0,0.3370,0.02690,...,0.0864,0.390,103.827,audio_features,1Bt06pqlg8pqlpnl4R6Rd2,spotify:track:1Bt06pqlg8pqlpnl4R6Rd2,https://api.spotify.com/v1/tracks/1Bt06pqlg8pq...,https://api.spotify.com/v1/audio-analysis/1Bt0...,177693,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Sand in My Shoes,Dido,4ObePnl5sf02pKfYUeGpkp,0.633,0.809,11,-6.702,1,0.0317,0.11400,...,0.1110,0.267,125.953,audio_features,4ObePnl5sf02pKfYUeGpkp,spotify:track:4ObePnl5sf02pKfYUeGpkp,https://api.spotify.com/v1/tracks/4ObePnl5sf02...,https://api.spotify.com/v1/audio-analysis/4Obe...,299173,4
190,Turn Me On,Norah Jones,1qzHqfvKrZWo6dVHM1dXrj,0.429,0.180,10,-10.694,1,0.0551,0.86400,...,0.1040,0.485,170.645,audio_features,1qzHqfvKrZWo6dVHM1dXrj,spotify:track:1qzHqfvKrZWo6dVHM1dXrj,https://api.spotify.com/v1/tracks/1qzHqfvKrZWo...,https://api.spotify.com/v1/audio-analysis/1qzH...,154800,3
191,Made You Look,Meghan Trainor,0QHEIqNKsMoOY5urbzN48u,0.838,0.525,10,-3.562,1,0.0665,0.34500,...,0.0771,0.884,144.981,audio_features,0QHEIqNKsMoOY5urbzN48u,spotify:track:0QHEIqNKsMoOY5urbzN48u,https://api.spotify.com/v1/tracks/0QHEIqNKsMoO...,https://api.spotify.com/v1/audio-analysis/0QHE...,134256,4
192,2 Be Loved (Am I Ready),Lizzo,13Irp51zj01BZu2XtDrnAg,0.721,0.769,7,-4.111,1,0.1050,0.09220,...,0.0817,0.915,155.932,audio_features,13Irp51zj01BZu2XtDrnAg,spotify:track:13Irp51zj01BZu2XtDrnAg,https://api.spotify.com/v1/tracks/13Irp51zj01B...,https://api.spotify.com/v1/audio-analysis/13Ir...,187108,4


In [39]:
playlist_features['analysis_url'][2]


'https://api.spotify.com/v1/audio-analysis/4gJAWJnE5cvAAWyTMTxgdY'